# Queue ADT 
---
## Design

Stack Methods

1. `Q.enqueue(e)` Add element e to the back of the queue Q.
    
2. `Q.dequeue()` Remove and return the first element from queue. An error occurs if queue is empty
    
Additional Accessor methods for convenience

3. `Q.first()` Return a reference to front element [NO REMOVAL]. empty Q gives error.

4. `Q.is_empty()` Return True if queue Q does not contain any elements.
    
5. `len(Q)` Return the number of elements in queue Q

Additional instance variables

6. `_data` reference to a list instance with a fixed capacity

7. `_size` current number of elements stored in the queue. NOT len(alist)

8. `_front` Index of first element of queue if it's not empty

---

## Array-based Queue Implementation

- Direct use of Lists as underlying storage is inefficient. 
    - Every `pop(0)` for dequeue needs to shift elements index left by one. Always worst case behaviour Big-Theta(n)

- Replace dequeued position with `None`. Maintain variable to store index of front
    - list grows with total no. enqueue operation. NOT a/c to actual no. elements in list.
    - in due time, queue would have relatively few elements, yet which are stored in an arbitrarily large list.

**Circular-Array Queue: [Better Approach]**

- When we dequeue an element and want to “advance” the front index, we use the arithmetic f = (f + 1) % N. 
- [here len(alist)=N means index from 0 - (N-1)]

- use of the modular arithmetic achieves the desired circular semantics.

-  For example, if our hypothetical queue had 3 elements with the first at index 8, our computation of (8+3) % 10 evaluates to 1, which is perfect since the three existing elements occupy indices 8, 9, and 0.



In [2]:
class Empty(Exception):
    pass

class ArrayQueue:
    """FIFO queue implementation with list"""
    
    def __init__(self, capacity=10): # default capacity 10
        """create empty queue"""
        self._data = [None]*capacity
        self._size = 0
        self._front = 0
        
    def __len__(self):
        """Return no. elements in queue"""
        return self._size
    
    def is_empty(self):
        return self._size==0
    
    def first(self):
        """Return (but donot remove) element at front of queue"""
        if self.is_empty():
            raise Empty('Empty Queue.')
        return self._data[self._front]
    
    def dequeue(self):
        """Remove and Return front element FIFO"""
        if self.is_empty():
            raise Empty('Empty queue')
        
        answer = self._data[self._front]
        
        self._data[self._front] = None  # help garbage collection 
        
        self._front = (self._front+1) % len(self._data)  # initialised queue length = capacity
        # NOT len(self) which is actual no. elements
        
        self._size = self._size - 1  # decrease queue size
        
        if 0 < self._size < len(self._data)//4 :
            self._resize(len(self._data)//2)
            """reduce the array to half of its current size, whenever the number of elements
                stored in it falls below one fourth of its capacity. a desirable property to 
                have queue space usage worst case Big-Theta(n)"""
            
        return answer
    
    def enqueue(self,value):
        """Add element to back of queue"""
        if self._size == len(self._data):
            self._resize(2*len(self._data)) # double array size if full
            
        available = (self._front + self._size) % len(self._data) # Modular operation for circular array
        self._data[available] = value
        self._size = self._size + 1
        return value, 'enqueue success'

    def _resize(self,capacity):
        """Resize to new list capacity which is greater than current size i.e.len(self)"""
        
        old = self._data  # keep track of existing list
        
        self._data = [None]*capacity  # allocate same list but with new capacity full of None
        
        walk = self._front # start walking from first element to all elements of queue
        
        for k in range(self._size): # consider all existing old elements
            self._data[k] = old[walk] # start newlist[0]<--first     newlist[1]<--- (first+1)%oldsize
            # no matter which index (first) in old list, it is copied to starting index[0] of new list
            walk = (walk+1)%len(old) # traverse old list circularly as it was designed 
            
        self._front = 0  # front in new list 
    
    def capacity(self):  # for debugging _resize
        return len(self._data)

In dequeue() method :

self. data[self. front] = None. 

Reason for the assignment to None relates to Python’s mechanism for reclaiming unused space. Internally, Python maintains a count of the number of references that exist to each object. If that count reaches zero, the object is effectively inaccessible, thus the system may reclaim that memory for future use.

The second significant responsibility of the dequeue method is to update the value of front to reflect the removal of the element, and the presumed promotion of the second element to become the new first.

```
In _resize :

While transferring the contents, we intentionally realign the front of the queue with index 0 in the new array.

 0  1  2  3  4  5
[H][I][J][E][F][G]
 -  -  -  f  -  -
 
[E][F][G][H][I][J][][][][][][][] New_Size --- Mod_change --- NO_exact_copy_indices_elements
 f  -  -  -  -  -
 ```

In [3]:

if __name__=='__main__':
    a = ArrayQueue(5)
    for i in range(1,15,3):
        print(a.enqueue(i))
        
    print('queue size',len(a))
    
    print('\nqueue capacity',a.capacity())
    
    print(a.enqueue('hi'))
    
    print('queue capacity',a.capacity())
    
    print('\nqueue first element',a.first())
    
    for i in range(1,18,3):
        print(f"Dequeue {a.dequeue()}   EmptyQueue {a.is_empty()}")
    
    a.dequeue() # error occurs
    

(1, 'enqueue success')
(4, 'enqueue success')
(7, 'enqueue success')
(10, 'enqueue success')
(13, 'enqueue success')
queue size 5

queue capacity 5
('hi', 'enqueue success')
queue capacity 10

queue first element 1
Dequeue 1   EmptyQueue False
Dequeue 4   EmptyQueue False
Dequeue 7   EmptyQueue False
Dequeue 10   EmptyQueue False
Dequeue 13   EmptyQueue False
Dequeue hi   EmptyQueue True


Empty: Empty queue